![](ubc_header.png)

# Climate Friendly Food System (CFFS) Labelling Pilot
***

## Objective

Implement the Climate-Friendly and Just Food System (CFFS &Just) definition at the UBC Campus by producing the weighted metric that informs the choice of icon for each menu item.

## Data Source

**Recipes Data**: extracted from UBC culinary services management platform Optimum Control

**cool_food_ghg.csv**: extracted from the [Cool Food Calculator](https://coolfood.org/pledge/). For this project, we use emission factors for the active total supply chain emissions in the North America region. The food category is at the detailed level for higher results accuracy

**ingredient_list.csv**: all ingredients appeared in the top 10 selling menu items with their associated food category in the Cool Food Category

## Assumptions

* The same GHG emission factor will be assigned to different forms (puree, sliced, chopped etc) of the same raw ingredient

* We assume the GHG factors for different varieties of one ingredient are the same (i.e Red/Yellow Onion)

* The loss/add of ingredient weight during the cooking process will be ignored

* The amount of unusable part and wastes during ingredients processing process are ignored 

* Ignore GHG emissions from the cooking process

* Assume the GHG emission factor for water is zero, and ignore the water use in the cooking process


## Import Libraries

In [1]:
#!pip3 install pdpipe

In [2]:
import numpy as np 
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt

import csv
from itertools import islice

## Load Data

#### Load Emission Factors

In [4]:
cfc_filename = '/Users/silvia/cffs-label/data/raw/cool_food_ghg.csv'
cfc = pd.read_csv(cfc_filename)
cfc.head()

,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,beef & buffalo meat,41.3463
1,lamb/mutton & goat meat,41.6211
2,pork (pig meat),9.8315
3,"poultry (chicken, turkey)",4.3996
4,butter,11.4316


In [5]:
cfc.shape

(54, 2)

#### Load Ingredient List

In [6]:
in10_filename = '/Users/silvia/cffs-label/data/raw/ingredient_list.csv'
in10 = pd.read_csv(in10_filename)
in10.head()

,Name of Ingredient,Food Category in the Cool Food Pledge
0,Black Beans|Drained,beans and pulses (dried)
1,Butter|Unsalted,butter
2,Arugula,cabbages and other brassicas (broccoli)
3,Cabbage,cabbages and other brassicas (broccoli)
4,Cauliflower Florets,cabbages and other brassicas (broccoli)


In [7]:
in10.shape

(116, 2)

## Data Cleaning

#### Construct Dictionary for Food Categories and Emission Factors

In [8]:
def row_csvtodict(csv_file):
    dict_club={}
    with open(csv_file)as f:
        reader = csv.reader(f,delimiter=',', )
        headers = next(reader)
        for row in reader:
            dict_club[row[0]]=row[1]
    return dict_club

row_csvtodict(cfc_filename)

{'beef & buffalo meat': '41.3463',
 'lamb/mutton & goat meat': '41.6211',
 'pork (pig meat)': '9.8315',
 'poultry (chicken, turkey)': '4.3996',
 'butter': '11.4316',
 'cheese': '8.9104',
 'ice cream': '4.0163',
 'cream': '6.9824',
 "milk (cow's milk)": '2.2325',
 'yogurt': '2.9782',
 'eggs': '3.6615',
 'fish (finfish)': '4.9798',
 'crustaceans (shrimp/prawns)': '21.1274',
 'mollusks': '2.4351',
 'animal fats': '6.9693',
 'other legumes': '1.6042',
 'beans and pulses (dried)': '1.6776',
 'peas': '0.6995',
 'peanuts/groundnuts': '1.692',
 'soybeans/tofu': '1.7542',
 'other grains/cereals': '1.4785',
 'corn (maize) ': '0.9734',
 'oats (oatmeal)': '2.3017',
 'wheat/rye (bread, pasta, baked goods)': '1.5225',
 'rice': '2.5345',
 'tree nuts and seeds': '4.2854',
 'almond milk': '0.7021',
 'oat milk': '0.9943',
 'rice milk': '0.6972',
 'soy milk': '0.489',
 'other fruits': '0.4306',
 'apples': '0.3581',
 'bananas': '0.7115',
 'berries': '1.6547',
 'citrus fruit ': '0.3942',
 'cabbages and oth

## Data Analysis

#### Mapping Ingredients to GHG Factors (For the top 10 menu items)

In [9]:
import openpyxl

df1 = pd.DataFrame(in10)
df2 = pd.DataFrame(cfc)

In [10]:
df1

,Name of Ingredient,Food Category in the Cool Food Pledge
0,Black Beans|Drained,beans and pulses (dried)
1,Butter|Unsalted,butter
2,Arugula,cabbages and other brassicas (broccoli)
3,Cabbage,cabbages and other brassicas (broccoli)
4,Cauliflower Florets,cabbages and other brassicas (broccoli)
...,...,...
111,Sauce|Chili Hot Sriracha,NaN
112,Sauce|Sweet & Sour,NaN
113,Sauce|Sweet Chili,NaN
114,Sauce|Worchestershire,NaN


In [11]:
df2

,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,beef & buffalo meat,41.3463
1,lamb/mutton & goat meat,41.6211
2,pork (pig meat),9.8315
3,"poultry (chicken, turkey)",4.3996
4,butter,11.4316
5,cheese,8.9104
6,ice cream,4.0163
7,cream,6.9824
8,milk (cow's milk),2.2325
9,yogurt,2.9782


In [12]:
result = pd.merge(df1, df2.loc[:,['Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'Food Category in the Cool Food Pledge', 
                  right_on = 'Food Category')
result

,Name of Ingredient,Food Category in the Cool Food Pledge,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,Black Beans|Drained,beans and pulses (dried),beans and pulses (dried),1.6776
1,Butter|Unsalted,butter,butter,11.4316
2,Arugula,cabbages and other brassicas (broccoli),cabbages and other brassicas (broccoli),0.6220
3,Cabbage,cabbages and other brassicas (broccoli),cabbages and other brassicas (broccoli),0.6220
4,Cauliflower Florets,cabbages and other brassicas (broccoli),cabbages and other brassicas (broccoli),0.6220
...,...,...,...,...
111,Sauce|Chili Hot Sriracha,NaN,NaN,NaN
112,Sauce|Sweet & Sour,NaN,NaN,NaN
113,Sauce|Sweet Chili,NaN,NaN,NaN
114,Sauce|Worchestershire,NaN,NaN,NaN
